# Fine tune Flan T5 model using the Codeflare stack and Ray distribution

This notebook fine tunes the flan T5 model with a summarization dataset. It first uses Instascale to add required machines to the Openshift cluster and then uses Codeflare stack to spawn up a ray cluster. Then it uses Ray train api to distribute the training job over multiple nodes.

In [1]:
# Import pieces from codeflare-sdk
from codeflare_sdk.cluster.cluster import Cluster, ClusterConfiguration
from codeflare_sdk.cluster.auth import TokenAuthentication

In [1]:
# Check ray version: it should match the worker's ray version
import ray
ray.__version__

'2.1.0'

In [2]:
# Create authentication object for oc user permissions
auth = TokenAuthentication(
    token = "xxx",
    server = "https://api.et-cluster.6mwp.p1.openshiftapps.com:6443",
    skip_tls=False
)
auth.login()

'Logged into "https://api.et-cluster.6mwp.p1.openshiftapps.com:6443" as "shanand@redhat.com" using the token provided.\n\nYou have access to 113 projects, the list has been suppressed. You can list all projects with \'oc projects\'\n\nUsing project "opendatahub".\n'

Once again, let's start by running through the same cluster setup as before:

In [3]:
# Create and configure our cluster object (and appwrapper)
cluster = Cluster(ClusterConfiguration(
    name='finetuneflan',
    namespace='default',
    min_worker=2,
    max_worker=2,
    min_cpus=1,
    max_cpus=2,
    min_memory=8,
    max_memory=24,
    gpu=1,
    instascale=True,
    machine_types=["m5.xlarge", "g4dn.xlarge"]
))

Written to: finetuneflan.yaml


In [14]:
# Bring up the cluster
cluster.up()
cluster.wait_ready()

Waiting for requested resources to be set up...


KeyboardInterrupt: 

In [15]:
cluster.details()

                     🚀 CodeFlare Cluster Details 🚀                     
                                                                         
 ╭─────────────────────────────────────────────────────────────────────╮ 
 │   Name                                                              │ 
 │   finetuneflan                                        Inactive ❌   │ 
 │                                                                     │ 
 │   URI: ray://finetuneflan-head-svc.default.svc:10001                │ 
 │                                                                     │ 
 │   ]8;id=384441;http://ray-dashboard-finetuneflan-default.apps.et-cluster.6mwp.p1.openshiftapps.com\Dashboard🔗]8;;\                                                       │ 
 │                                                                     │ 
 │                      Cluster Resources                              │ 
 │   ╭─ Workers ──╮  ╭───────── Worker specs(each) ─────────╮          │ 
 │   │  Min  Max  │  │  Memory      CPU         GPU         │          │ 
 │   │            │  │                                      │          │ 
 │   │  2    2    │  │  8~24        1           1           │          │ 
 │   │            │  │                                      │          │ 
 │   ╰────────────╯  ╰──────────────────────────────────────╯          │ 
 ╰─────────────────────────────────────────────────────────────────────╯

RayCluster(name='finetuneflan', status=<CodeFlareClusterStatus.STARTING: 2>, min_workers=2, max_workers=2, worker_mem_min=8, worker_mem_max=24, worker_cpu=1, worker_gpu=1, namespace='default', dashboard='http://ray-dashboard-finetuneflan-default.apps.et-cluster.6mwp.p1.openshiftapps.com')

This time we will demonstrate another potential method of use: working with the Ray cluster interactively.

Using the SDK, we can get both the Ray cluster URI and dashboard URI:

In [5]:
ray_dashboard_uri = cluster.cluster_dashboard_uri()
ray_cluster_uri = cluster.cluster_uri()
print(ray_dashboard_uri)
print(ray_cluster_uri)

http://ray-dashboard-finetuneflan-default.apps.et-cluster.6mwp.p1.openshiftapps.com
ray://finetuneflan-head-svc.default.svc:10001


Now we can connect directly to our Ray cluster via the Ray python client:

In [6]:
#before proceeding make sure the cluster exists and the uri is not empty
assert ray_cluster_uri, "Ray cluster needs to be started and set before proceeding"

import ray
from ray.air.config import ScalingConfig

# reset the ray context in case there's already one. 
ray.shutdown()
# establish connection to ray cluster

#install additionall libraries that will be required for model training
runtime_env = {"pip": ["transformers",
                       "datasets",
                       "evaluate",
                       "pyarrow<7.0.0",
                       "accelerate",
                       "loralib",
                       "py7zr",
                       "tensorboard",
                       "peft"], 
              "env_vars": {"HF_HOME":"huggingface"}}

ray.init(address=f'{ray_cluster_uri}', runtime_env=runtime_env, _temp_dir="huggingface")

print("Ray cluster is up and running: ", ray.is_initialized())

Ray cluster is up and running:  True


Now that we are connected (and have passed in some package requirements), let's try writing some training code for a DistilBERT transformer model via HuggingFace (using IMDB dataset):

In [7]:
@ray.remote
def train_fn():
    from datasets import load_dataset
    import transformers
    from transformers import AutoTokenizer, TrainingArguments
    from transformers import AutoModelForSequenceClassification
    import numpy as np
    from datasets import load_metric
    import ray
    from ray import tune
    from ray.train.huggingface import HuggingFaceTrainer
    from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
    from datasets import load_dataset, concatenate_datasets
    from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
    from peft import LoraConfig, get_peft_model, TaskType #, prepare_model_for_int8_training

    model_name = "google/flan-t5-xl"

    #model = AutoModelForSeq2SeqLM.from_pretrained(model_name, load_in_8bit=True, device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    dataset = load_dataset("samsum")

    print(f"Train dataset size: {len(dataset['train'])}")
    print(f"Test dataset size: {len(dataset['test'])}")
    
    #### COMPUTE MAX SEQ LEN ##########
    # The maximum total input sequence length after tokenization.
    # Sequences longer than this will be truncated, sequences shorter will be padded.
    conc_dataset = concatenate_datasets([dataset["train"], dataset["test"]])

    
    tokenized_inputs = conc_dataset.map(lambda x: tokenizer(x["dialogue"],
                                                            truncation=True),
                                        batched=True,
                                        remove_columns=["dialogue", "summary"])
    
    input_lengths = [len(x) for x in tokenized_inputs["input_ids"]]
    # take 85 percentile of max length for better utilization
    max_source_length = int(np.percentile(input_lengths, 85))
    print(f"Max source length: {max_source_length}")

    # The maximum total sequence length for target text after tokenization.
    # Sequences longer than this will be truncated, sequences shorter will be padded."
    tokenized_targets = conc_dataset.map(lambda x: tokenizer(x["dialogue"],
                                                            truncation=True),
                                        batched=True,
                                        remove_columns=["dialogue", "summary"])  
    target_lengths = [len(x) for x in tokenized_targets["input_ids"]]
    # take 90 percentile of max length for better utilization
    max_target_length = int(np.percentile(target_lengths, 90))
    print(f"Max target length: {max_target_length}")
    
    #### PREPROCESS DATA ##########
    
    def preprocess_function(sample,padding="max_length"):
        # add prefix to the input for t5
        inputs = ["summarize: " + item for item in sample["dialogue"]]

        # tokenize inputs
        model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

        # Tokenize targets with the `text_target` keyword argument
        labels = tokenizer(text_target=sample["summary"], max_length=max_target_length, padding=padding, truncation=True)

        # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
        # padding in the loss.
        if padding == "max_length":
            labels["input_ids"] = [
                [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
            ]

        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["dialogue", "summary", "id"])
    print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

    ray_train_ds = ray.data.from_huggingface(tokenized_dataset['train'])
    ray_evaluation_ds = ray.data.from_huggingface(tokenized_dataset['test'])

    def compute_metrics(eval_pred):
        metric = load_metric("accuracy")
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels)
    
    def trainer_init_per_worker(train_dataset, eval_dataset, **config):
        model_name = "google/flan-t5-xl"
        model = AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map="auto")
        lora_config = LoraConfig(
            r=16,
            lora_alpha=32,
            target_modules=["q", "v"],
            lora_dropout=0.05,
            bias="none",
            task_type=TaskType.SEQ_2_SEQ_LM
        )
        # prepare int-8 model for training
        #model = prepare_model_for_int8_training(model)

        # add LoRA adaptor
        model = get_peft_model(model, lora_config)
        model.print_trainable_parameters()
        
        from transformers import DataCollatorForSeq2Seq

        # we want to ignore tokenizer pad token in the loss
        label_pad_token_id = -100
        # Data collator
        data_collator = DataCollatorForSeq2Seq(
            tokenizer,
            model=model,
            label_pad_token_id=label_pad_token_id,
            pad_to_multiple_of=8
        )
        
        output_dir="/tmp/flan/test"

        # Define training args
        training_args = Seq2SeqTrainingArguments(
            output_dir=output_dir,
            auto_find_batch_size=True,
            learning_rate=1e-3, # higher learning rate
            num_train_epochs=5,
            logging_dir=f"{output_dir}/logs",
            logging_strategy="steps",
            logging_steps=500,
            save_strategy="no",
            report_to="tensorboard",
        )

        trainer = Seq2SeqTrainer(model=model,
                                args=training_args,
                                data_collator=data_collator,
                                train_dataset=train_dataset,
                                eval_dataset=eval_dataset)
        
        return trainer

    scaling_config = ScalingConfig(num_workers=2, use_gpu=True) #num workers is the number of gpus

    # we are using the ray native HuggingFaceTrainer, but you can swap out to use non ray Huggingface Trainer. Both have the same method signature. 
    # the ray native HFTrainer has built in support for scaling to multiple GPUs
    trainer = HuggingFaceTrainer(
        trainer_init_per_worker=trainer_init_per_worker,
        scaling_config=scaling_config,
        datasets={"train": ray_train_ds, "evaluation": ray_evaluation_ds},
    )
    result = trainer.fit()
    return result

Once we want to test our code out, we can run the training function we defined above remotely on our Ray cluster:

In [8]:
#call the above cell as a remote ray function
r = ray.get(train_fn.remote())

(train_fn pid=4614) Train dataset size: 14732
(train_fn pid=4614) Test dataset size: 819


(train_fn pid=4614) Found cached dataset samsum (/home/ray/workspace/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e)
Map:  96%|█████████▋| 15000/15551 [00:03<00:00, 4321.68 examples/s]
                                                                   


(train_fn pid=4614) Max source length: 255


Map:  96%|█████████▋| 15000/15551 [00:03<00:00, 4252.76 examples/s]
                                                                   


(train_fn pid=4614) Max target length: 297


...

(train_fn pid=4614) == Status ==
(train_fn pid=4614) Current time: 2023-07-27 08:01:44 (running for 00:03:23.36)
(train_fn pid=4614) Memory usage on this node: 3.9/15.4 GiB 
(train_fn pid=4614) Using FIFO scheduling algorithm.
(train_fn pid=4614) Resources requested: 1.0/6 CPUs, 2.0/2 GPUs, 0.0/52.15 GiB heap, 0.0/11.29 GiB objects
(train_fn pid=4614) Result logdir: /home/ray/ray_results/HuggingFaceTrainer_2023-07-27_07-58-20
(train_fn pid=4614) Number of trials: 1/1 (1 RUNNING)
(train_fn pid=4614) +--------------------------------+----------+------------------+
(train_fn pid=4614) | Trial name                     | status   | loc              |
(train_fn pid=4614) |--------------------------------+----------+------------------|
(train_fn pid=4614) | HuggingFaceTrainer_06f2f_00000 | RUNNING  | 10.128.30.22:345 |
(train_fn pid=4614) +--------------------------------+----------+------------------+
(train_fn pid=4614) 
(train_fn pid=4614) 
(train_fn pid=4614) == Status ==
(train_fn pid=46

A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: ffffffffffffffff537034a8f0299b4acc1e1f4e05000000 Worker ID: 20fd7164328a7217f296071d2bde261324cae0cd44aa1a5887306bbd Node ID: 37f6f748268c35754f1bf5790acbd3c6a9f245776da67ebc9830fc1d Worker IP address: 10.128.30.22 Worker port: 10007 Worker PID: 378 Worker exit type: SYSTEM_ERROR Worker exit detail: Worker unexpectedly exits with a connection error code 2. End of file. There are some potential root causes. (1) The process is killed by SIGKILL by OOM killer due to high memory usage. (2) ray stop --force is called. (3) The worker is crashed unexpectedly due to SIGSEGV or other unexpected errors.
(HuggingFaceTrainer pid=345, ip=10.128.30.22) 2023-07-27 08:02:12,849	INFO utils.py:57 -- Worker 1 has failed.


(train_fn pid=4614) Result for HuggingFaceTrainer_06f2f_00000:
(train_fn pid=4614)   date: 2023-07-27_07-58-29
(train_fn pid=4614)   experiment_id: 182233fc8cb24d72bfa113cb6a3f25bd
(train_fn pid=4614)   hostname: finetuneflan-worker-small-group-finetuneflan-6f6ft
(train_fn pid=4614)   node_ip: 10.128.30.22
(train_fn pid=4614)   pid: 345
(train_fn pid=4614)   timestamp: 1690469909
(train_fn pid=4614)   trial_id: 06f2f_00000
(train_fn pid=4614)   
(train_fn pid=4614) == Status ==
(train_fn pid=4614) Current time: 2023-07-27 08:02:13 (running for 00:03:52.79)
(train_fn pid=4614) Memory usage on this node: 3.9/15.4 GiB 
(train_fn pid=4614) Using FIFO scheduling algorithm.
(train_fn pid=4614) Resources requested: 0/6 CPUs, 0/2 GPUs, 0.0/52.15 GiB heap, 0.0/11.29 GiB objects
(train_fn pid=4614) Result logdir: /home/ray/ray_results/HuggingFaceTrainer_2023-07-27_07-58-20
(train_fn pid=4614) Number of trials: 1/1 (1 ERROR)
(train_fn pid=4614) +--------------------------------+----------+-------

(train_fn pid=4614) 2023-07-27 08:02:13,669	ERROR trial_runner.py:993 -- Trial HuggingFaceTrainer_06f2f_00000: Error processing event.
(train_fn pid=4614) ray.exceptions.RayTaskError(RuntimeError): ray::_Inner.train() (pid=345, ip=10.128.30.22, repr=HuggingFaceTrainer)
(train_fn pid=4614) ray.exceptions.RayActorError: The actor died unexpectedly before finishing this task.
(train_fn pid=4614) 	class_name: RayTrainWorker
(train_fn pid=4614) 	actor_id: 537034a8f0299b4acc1e1f4e05000000
(train_fn pid=4614) 	pid: 378
(train_fn pid=4614) 	namespace: 79e19797-9a9d-4359-9e7e-135e143c02c0
(train_fn pid=4614) 	ip: 10.128.30.22
(train_fn pid=4614) The actor is dead because its worker process has died. Worker exit type: SYSTEM_ERROR Worker exit detail: Worker unexpectedly exits with a connection error code 2. End of file. There are some potential root causes. (1) The process is killed by SIGKILL by OOM killer due to high memory usage. (2) ray stop --force is called. (3) The worker is crashed unexp

RayTaskError(RuntimeError): [36mray::train_fn()[39m (pid=4614, ip=10.128.28.7)
  File "/tmp/ipykernel_14249/2624701892.py", line 150, in train_fn
  File "/home/ray/anaconda3/lib/python3.8/site-packages/ray/train/base_trainer.py", line 360, in fit
    raise result.error
ray.exceptions.RayTaskError(RuntimeError): [36mray::_Inner.train()[39m (pid=345, ip=10.128.30.22, repr=HuggingFaceTrainer)
ray.exceptions.RayActorError: The actor died unexpectedly before finishing this task.
	class_name: RayTrainWorker
	actor_id: 537034a8f0299b4acc1e1f4e05000000
	pid: 378
	namespace: 79e19797-9a9d-4359-9e7e-135e143c02c0
	ip: 10.128.30.22
The actor is dead because its worker process has died. Worker exit type: SYSTEM_ERROR Worker exit detail: Worker unexpectedly exits with a connection error code 2. End of file. There are some potential root causes. (1) The process is killed by SIGKILL by OOM killer due to high memory usage. (2) ray stop --force is called. (3) The worker is crashed unexpectedly due t

In [ ]:
!pip install --upgrade ray peft accelerate

In [ ]:
from ray.train.huggingface.transformers.transformers_checkpoint import TransformersCheckpoint
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import PeftModel, PeftConfig

model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-large')
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-large')

checkpoint = TransformersCheckpoint.from_checkpoint(r.checkpoint)

model_output_dir = '../../models/raytune'
checkpoint.to_directory(model_output_dir)

# Load the Lora model
model = PeftModel.from_pretrained(model, model_output_dir)

Once complete, we can bring our Ray cluster down and clean up:

In [7]:
# To do next:
# - train on ROSA data and add inference code
# - train a higher param model
# - Add bitsandbytes

In [12]:
cluster.down()

In [ ]:
auth.logout()